In [1]:

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Flatten
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

sentences = [
    "the cat sat on the mat",
    "the dog sat on the log",
    "cats and dogs are great pets",
    "dogs are better than cats",
    "the mat is on the floor"
]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
total_words = len(tokenizer.word_index) + 1  # Adding 1 for padding

print("Total Words:", total_words)


def create_cbow_pairs(sentences, window_size=2):
    input_data = []
    output_data = []

    for sentence in sentences:
        words = sentence.split()
        for i, word in enumerate(words):
            # Define the context window
            start_index = max(0, i - window_size)
            end_index = min(len(words), i + window_size + 1)
            context = [words[j] for j in range(start_index, end_index) if j != i]

            # Append the context and target word
            input_data.append(context)
            output_data.append(word)

    return input_data, output_data


input_data, output_data = create_cbow_pairs(sentences)

input_sequences = tokenizer.texts_to_sequences(input_data)
output_sequences = tokenizer.texts_to_sequences(output_data)

# Convert output to one-hot encoding
output_sequences = tf.keras.utils.to_categorical(output_sequences, num_classes=total_words)

max_length = max(len(seq) for seq in input_sequences)
input_sequences = pad_sequences(input_sequences, maxlen=max_length, padding='post')

model = Sequential()
model.add(Embedding(input_dim=total_words, output_dim=10))  # Embedding layer
model.add(Flatten())  # Flatten the output of embedding layer
model.add(Dense(total_words, activation='softmax'))  # Output layer

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(input_sequences, output_sequences, epochs=100, verbose=1)


# Function to predict word from context
def predict_word(context):
    context_seq = tokenizer.texts_to_sequences([context])
    context_seq = pad_sequences(context_seq, maxlen=max_length, padding='post')
    predicted = model.predict(context_seq)
    return tokenizer.index_word[np.argmax(predicted)]


# Example prediction
context_example = ["the", "cat", "on", "the"]
predicted_word = predict_word(context_example)
print(f"Predicted word for context {context_example}: {predicted_word}")



Total Words: 18
Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 541ms/step - accuracy: 0.0690 - loss: 2.8939
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.1034 - loss: 2.8901
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.0690 - loss: 2.8864
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.0690 - loss: 2.8826
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.0690 - loss: 2.8789
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.1034 - loss: 2.8752
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.1034 - loss: 2.8714
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.1724 - loss: 2.8677
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.2414 - loss: 2.8640
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.2759 - loss: 2.8602
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.3103 - loss: 2.8565
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accu